In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Load cryptocurrency price data from CSV file
df = pd.read_csv('crypto_prices.csv')

# Scale the data between 0 and 1
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))

# Split the data into training and testing sets
training_size = int(len(scaled_data) * 0.7)
test_size = len(scaled_data) - training_size
train_data, test_data = scaled_data[0:training_size,:], scaled_data[training_size:len(scaled_data),:]

# Define the window size for LSTM network
window_size = 60

# Define the training and testing data for LSTM network
def create_dataset(dataset, window_size):
    data_X, data_Y = [], []
    for i in range(len(dataset)-window_size-1):
        a = dataset[i:(i+window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return np.array(data_X), np.array(data_Y)

train_X, train_Y = create_dataset(train_data, window_size)
test_X, test_Y = create_dataset(test_data, window_size)

# Define the LSTM network model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(train_X.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_X, train_Y, epochs=50, batch_size=32, verbose=2)

# Make predictions on test data
test_predictions = model.predict(test_X)

# Inverse transform the predictions and actual values to get the original scale
test_predictions = scaler.inverse_transform(test_predictions)
test_Y = scaler.inverse_transform(test_Y.reshape(-1, 1))

# Plot the predicted and actual prices
plt.plot(test_Y, label='Actual Price')
plt.plot(test_predictions, label='Predicted Price')
plt.legend()
plt.show()
